In [9]:
import json 
import urllib.request
from urllib import parse
from lxml import etree

import sqlalchemy
from sqlalchemy import create_engine, Integer, String, ForeignKey, MetaData, Text, Column, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [10]:
url = "http://kipo-api.kipi.or.kr/openapi/service/trademarkInfoSearchService/getAdvancedSearch?application=true&registration=true&refused=true&expiration=true&withdrawal=true&publication=true&cancel=true&abandonment=true&trademark=true&serviceMark=true&trademarkServiceMark=true&businessEmblem=true&collectiveMark=true&internationalMark=true&character=true&figure=true&compositionCharacter=true&figureComposition=true&freeSearch=?&ServiceKey=SbSvsMbb1IC21z3j8aPLAbHu%2Ff7CBZhA8WpulwUWOmzyThkPoM75Pg2CLYki4McA3AMslp8MtrJaVKVB1EOXRA%3D%3D&numOfRows=5000"

In [11]:
engine = create_engine("sqlite:///project2.db", echo = False)
base = declarative_base()
Session  = sessionmaker(bind=engine)
session = Session()

#스키마 선언
class Item(base):
    __tablename__ = "Item"
    __table_args__= {'extend_existing': True} 
    id = Column(Integer, primary_key = True)
    agentName = Column(String)
    appReferenceNumber = Column(String)
    applicantName = Column(Text)
    applicationDate = Column(String)
    applicationNumber = Column(String)
    applicationStatus = Column(String)
    bigDrawing = Column(String)
    classificationCode = Column(String)
    drawing = Column(String)
    fullText = Column(String)
    indexNo = Column(Integer)
    internationalRegisterDate = Column(String)
    internationalRegisterNumber = Column(String)
    priorityDate = Column(String)
    priorityNumber = Column(String)
    publicationDate = Column(String)
    publicationNumber = Column(String)
    regPrivilegeName = Column(String)
    regReferenceNumber = Column(String)
    registrationDate = Column(String)
    registrationNumber = Column(String)
    registrationPublicDate = Column(String)
    registrationPublicNumber = Column(String)
    title = Column(Text)
    viennaCode = Column(String)

base.metadata.create_all(engine)

In [12]:
def crawling(start_page=0,end_page=100):
    # range 범위로 몇페이지까지 할껀지 설정
    for i in range(start_page,end_page):
        f = open("log/log"+str(i),"w",encoding="utf-8") # 간단하게 로그 남기려고 맨듬
        parser = etree.XMLParser(encoding='utf-8' ,recover=True) # 파서 추가 
        new_url = url+"&pageNo="+str(i)
        req = urllib.request.Request(new_url)
        res = urllib.request.urlopen(req)
        resResult = res.read()
        xml = etree.fromstring(resResult,parser = parser) # 파서 적용
        xmlTree = etree.ElementTree(xml)
        root = xmlTree.getroot()
        items = root.findall(".//item")
        num = 0
        for item in items:
            new_item = Item(
                agentName = item[0].text,
                appReferenceNumber = item[1].text,
                applicantName = item[2].text,
                applicationDate = item[3].text,
                applicationNumber = item[4].text,
                applicationStatus = item[5].text,
                bigDrawing = item[6].text,
                classificationCode = item[7].text,
                drawing = item[8].text,
                fullText = item[9].text,
                indexNo = item[10].text,
                internationalRegisterDate = item[11].text,
                internationalRegisterNumber = item[12].text,
                priorityDate = item[13].text,
                priorityNumber = item[14].text,
                publicationDate = item[15].text,
                publicationNumber = item[16].text,
                regPrivilegeName = item[17].text,
                regReferenceNumber = item[18].text,
                registrationDate = item[19].text,
                registrationNumber = item[20].text,
                registrationPublicDate = item[21].text,
                registrationPublicNumber = item[22].text,
                title = item[23].text,
                viennaCode = item[24].text
            )
            f.write(str(num)+" "+item[4].text+"sucess \n")
            session.add(new_item)
            num+=1
        session.commit()
        f.close()

In [13]:
crawling(0,1)